In [ ]:
import numpy as np
import os

from app.environment.dataprovider import DataProvider
from app.preparation.preparator import DataPreparator

In [ ]:
apikey = os.getenv('TIINGO_API_KEY')
days = 5
coef_threshold = 0.98

train_start_date = '2000-01-01'
train_end_date = '2015-12-31'

test_start_date = '2016-01-01'
test_end_date = '2020-12-31'

provider = DataProvider(apikey)

In [ ]:
train_quotes = provider.load(train_ticker, train_start_date, train_end_date)
train_quotes['window'] = train_windows = \
    DataPreparator.calculate_windows(
        train_quotes,
        days=days,
        normalize=True,
        columns=['open', 'high', 'low', 'close'],
        adjust=provider.adjust_prices)
windows = np.array(train_quotes['window'].values.tolist())
shape = windows.shape
windows = windows.reshape((shape[0], shape[1] * shape[2] * shape[3]))
windows = [window for window in windows]
train_quotes['window'] = windows
train_quotes[['buy', 'sell']] = DataPreparator.calculate_signals(train_quotes)
train_quotes['prev_close'] = train_quotes['close'].shift(1).fillna(0.0)
train_quotes['next_close'] = train_quotes['close'].shift(-1).fillna(0.0)
train_quotes['prev_change'] = (train_quotes['close'] / train_quotes['prev_close']) - 1.0
train_quotes['next_change'] = (train_quotes['next_close'] / train_quotes['close']) - 1.0

In [ ]:
windows = train_quotes['window'].values[days - 1:].tolist()
samples = train_quotes[train_quotes['next_change'] > 0.02]['window'].values.tolist()
coefs = np.corrcoef(windows, samples, rowvar=True)
coefs_indices = np.where((coefs >= coef_threshold))
coefs_filter = (coefs_indices[0] != coefs_indices[1]) & (coefs_indices[0] < len(windows)) & (coefs_indices[1] >= len(windows))
coefs_indices = np.swapaxes(coefs_indices, 0, 1)
trade_indices = coefs_indices[coefs_filter, 0] + days - 1
sample_indices = coefs_indices[coefs_filter, 1] - len(windows)
results = train_quotes.iloc[trade_indices]['next_change'].values
train_samples = {}
for index in range(len(results)):
    sample_index = sample_indices[index]
    train_samples[sample_index] = (0 if sample_index not in train_samples else train_samples[sample_index]) + results[index]
trained_samples = [samples[sample_index] for sample_index, total_result in train_samples.items() if total_result > 0.02]
print(len(samples))
print(len(trained_samples))
# coefs = np.corrcoef(trained_samples, rowvar=True)
# coefs_indices = np.where((coefs >= coef_threshold))
# coefs_filter = (coefs_indices[0] != coefs_indices[1])
# coefs_indices = np.swapaxes(coefs_indices, 0, 1)
# trained_sample_indices = set(coefs_indices[coefs_filter, 0])
# trained_samples = [trained_samples[index] for index in trained_sample_indices]

In [ ]:
test_quotes = provider.load(test_ticker, test_start_date, test_end_date)
test_quotes['window'] = test_windows = \
    DataPreparator.calculate_windows(
        test_quotes,
        days=days,
        normalize=True,
        columns=['open', 'high', 'low', 'close'],
        adjust=provider.adjust_prices)
windows = np.array(test_quotes['window'].values.tolist())
shape = windows.shape
windows = windows.reshape((shape[0], shape[1] * shape[2] * shape[3]))
windows = [window for window in windows]
test_quotes['window'] = windows
test_quotes[['buy', 'sell']] = DataPreparator.calculate_signals(test_quotes)
test_quotes['prev_close'] = test_quotes['close'].shift(1).fillna(0.0)
test_quotes['next_close'] = test_quotes['close'].shift(-1).fillna(0.0)
test_quotes['prev_change'] = (test_quotes['close'] / test_quotes['prev_close']) - 1.0
test_quotes['next_change'] = (test_quotes['next_close'] / test_quotes['close']) - 1.0

In [ ]:
check_samples = trained_samples
check_quotes = train_quotes

windows = np.array(check_quotes['window'].values.tolist())
coefs = np.corrcoef(windows, check_samples, rowvar=True)
coefs_indices = np.where((coefs >= coef_threshold))
my_filter = (coefs_indices[0] != coefs_indices[1])  & (coefs_indices[0] < len(windows)) & (coefs_indices[1] >= len(windows))
coefs_indices = np.swapaxes(coefs_indices, 0, 1)
trade_indices = coefs_indices[my_filter, 0]
sample_indices = coefs_indices[my_filter, 1] - len(windows)
results = check_quotes.iloc[trade_indices]['next_change'].values

trades = [[results[index], sample_indices[index]] for index in range(len(results))]

total = 1.0
good = 0
bad = 0
for trade in trades:
    print(f'result {trade[0] * 100.0:5.2f} > sample {trade[1]}')
    if trade[0] > 0.0:
        trade[0] *= (0.25 * 1.055)
        good += 1
    else:
        bad += 1
    total *= 1.0 + trade[0]
print(f'capital: {100000.0 * total:.2f} / {good} : {bad} / {total:.2f}')